In [15]:
import geopandas as gpd
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [16]:

# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)

In [17]:
from constants import LIMIT, DATA_PATH, LIMIT_SUBSET

In [18]:
LIMIT_SUBSET

'C:\\Users\\bsf31\\Documents\\post-meds\\data\\policy-data\\preprocessing\\ready-dataset.gpkg'

In [19]:
'''# Load the shapefile using geopandas
limit = gpd.read_file(LIMIT)'''



'# Load the shapefile using geopandas\nlimit = gpd.read_file(LIMIT)'

In [20]:
'''limit['anho_capa'] = limit['anho_capa'].astype(int)

limit = limit[['id', 'put_id', 'anho_capa','fecha_res', 'geometry' ]]

filtered_limit = limit[limit['anho_capa'] >= 2000]
filtered_limit['area'] = filtered_limit['geometry'].area'''

"limit['anho_capa'] = limit['anho_capa'].astype(int)\n\nlimit = limit[['id', 'put_id', 'anho_capa','fecha_res', 'geometry' ]]\n\nfiltered_limit = limit[limit['anho_capa'] >= 2000]\nfiltered_limit['area'] = filtered_limit['geometry'].area"

In [21]:
'''# Sort properties by registration year
properties = filtered_limit.sort_values(by='anho_capa')

# Convert fecha_res to datetime format
properties['fecha_res'] = pd.to_datetime(properties['fecha_res'], errors='coerce')

# For rows with NaT (Not a Timestamp) in fecha_res, assign a default date based on their year
properties.loc[properties['fecha_res'].isna(), 'fecha_res'] = pd.to_datetime(properties['anho_capa'].astype(str) + '-01-01')

# Sort properties by fecha_res
properties = properties.sort_values(by='fecha_res')'''


"# Sort properties by registration year\nproperties = filtered_limit.sort_values(by='anho_capa')\n\n# Convert fecha_res to datetime format\nproperties['fecha_res'] = pd.to_datetime(properties['fecha_res'], errors='coerce')\n\n# For rows with NaT (Not a Timestamp) in fecha_res, assign a default date based on their year\nproperties.loc[properties['fecha_res'].isna(), 'fecha_res'] = pd.to_datetime(properties['anho_capa'].astype(str) + '-01-01')\n\n# Sort properties by fecha_res\nproperties = properties.sort_values(by='fecha_res')"

In [22]:
'''# Create an empty GeoDataFrame to store the final processed properties
final_properties = gpd.GeoDataFrame(columns=properties.columns)

# Add properties from the year 2000 to final_properties as the baseline
final_properties = pd.concat([final_properties, properties[properties['anho_capa'] == 2000]])

for year in range(2001, 2023):  # Loop from 2001 to 2022
    # Get properties of the current year
    current_year_properties = properties[properties['anho_capa'] == year]
    
    # Iterate over each property of the current year
    for idx, current_property in current_year_properties.iterrows():
        # Subtract geometries of older properties from the current property
        for _, older_property in final_properties.iterrows():
            current_property['geometry'] = current_property['geometry'].difference(older_property['geometry'])
        
        # Append the "cut" current property to the final_properties GeoDataFrame
        final_properties = pd.concat([final_properties, current_property.to_frame().T])
final_properties.crs = properties.crs'''




'# Create an empty GeoDataFrame to store the final processed properties\nfinal_properties = gpd.GeoDataFrame(columns=properties.columns)\n\n# Add properties from the year 2000 to final_properties as the baseline\nfinal_properties = pd.concat([final_properties, properties[properties[\'anho_capa\'] == 2000]])\n\nfor year in range(2001, 2023):  # Loop from 2001 to 2022\n    # Get properties of the current year\n    current_year_properties = properties[properties[\'anho_capa\'] == year]\n    \n    # Iterate over each property of the current year\n    for idx, current_property in current_year_properties.iterrows():\n        # Subtract geometries of older properties from the current property\n        for _, older_property in final_properties.iterrows():\n            current_property[\'geometry\'] = current_property[\'geometry\'].difference(older_property[\'geometry\'])\n        \n        # Append the "cut" current property to the final_properties GeoDataFrame\n        final_properties = pd.c

In [23]:
'''output_path = os.path.join(DATA_PATH,'preprocessing' "dataset-clip")

# Convert the 'fecha_res' column to a string format
final_properties['fecha_res'] = final_properties['fecha_res'].astype(str)

# Create the directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
    # Save the GeoDataFrame as a GeoPackage
# Define the filename for the GeoPackage

filename = os.path.join(output_path, "datset-clip.gpkg")
final_properties.to_file(filename, driver="GPKG")'''

'output_path = os.path.join(DATA_PATH,\'preprocessing\' "dataset-clip")\n\n# Convert the \'fecha_res\' column to a string format\nfinal_properties[\'fecha_res\'] = final_properties[\'fecha_res\'].astype(str)\n\n# Create the directory if it doesn\'t exist\nif not os.path.exists(output_path):\n    os.makedirs(output_path)\n    # Save the GeoDataFrame as a GeoPackage\n# Define the filename for the GeoPackage\n\nfilename = os.path.join(output_path, "datset-clip.gpkg")\nfinal_properties.to_file(filename, driver="GPKG")'

In [24]:
'''output_path = os.path.join(DATA_PATH, "limit-subsets")
# Create the directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

for year in sorted(filtered_limit['anho_capa'].unique()):
    year_gdf = filtered_limit[filtered_limit['anho_capa'] == year]
    
    # Define the filename for the GeoPackage
    filename = os.path.join(output_path, f"year_{year}.gpkg")
    
    # Save the GeoDataFrame as a GeoPackage
    year_gdf.to_file(filename, driver="GPKG")'''

'output_path = os.path.join(DATA_PATH, "limit-subsets")\n# Create the directory if it doesn\'t exist\nif not os.path.exists(output_path):\n    os.makedirs(output_path)\n\nfor year in sorted(filtered_limit[\'anho_capa\'].unique()):\n    year_gdf = filtered_limit[filtered_limit[\'anho_capa\'] == year]\n    \n    # Define the filename for the GeoPackage\n    filename = os.path.join(output_path, f"year_{year}.gpkg")\n    \n    # Save the GeoDataFrame as a GeoPackage\n    year_gdf.to_file(filename, driver="GPKG")'

In [25]:
'''# Create a list of GeoDataFrames for each year
year_gdfs = [gpd.read_file(os.path.join(LIMIT_SUBSETS[0], year_file))\
             for year_file in sorted(os.listdir(LIMIT_SUBSETS[0])) if year_file.endswith('.gpkg')]'''

"# Create a list of GeoDataFrames for each year\nyear_gdfs = [gpd.read_file(os.path.join(LIMIT_SUBSETS[0], year_file))             for year_file in sorted(os.listdir(LIMIT_SUBSETS[0])) if year_file.endswith('.gpkg')]"

In [26]:
# Load the shapefile using geopandas
limit_subset = gpd.read_file(LIMIT_SUBSET)

In [28]:
limit_subset

,id,put_id,anho_capa,fecha_res,area,geometry
0,176,PUT0176,2000,2000-01-19 00:00:00,5936099.973155426,"MULTIPOLYGON (((284617.950 7557184.826, 284608..."
1,511,PUT0511,2000,2000-02-01 00:00:00,5951123.782286445,"MULTIPOLYGON (((155454.161 7586494.623, 157619..."
2,467,PUT0467,2000,2000-02-10 00:00:00,47778603.62791812,"MULTIPOLYGON (((417515.525 7395323.775, 417870..."
3,512,PUT0512,2000,2000-03-07 00:00:00,3835414.179348219,"MULTIPOLYGON (((152259.745 7546016.649, 152183..."
4,169,PUT0169,2000,2000-03-07 00:00:00,10635728.4837288,"MULTIPOLYGON (((195978.190 7538496.754, 193320..."
...,...,...,...,...,...,...
3209,3261,PUT3261,2018,2018-02-01,90443701,"MULTIPOLYGON (((-23420.518 7565678.215, -21240..."
3210,3390,PUT3390,2020,2020-10-28,90507689,"MULTIPOLYGON (((318174.855 7815778.151, 318136..."
3211,3568,PUT3568,2022,2022-01-13,45633418,"MULTIPOLYGON (((52460.448 7501322.034, 52460.4..."
3212,3572,PUT3572,2021,2021-02-24,107722489,"MULTIPOLYGON (((123862.160 7596438.754, 123875..."
